# 추천시스템

- 코사인 유사도 기반 추천시스템
- 자카드 유사도 기반 추천시스템

개념 참조 사이트 : https://wooono.tistory.com/132

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

# 관광지 데이터 로드
df = pd.read_csv('해시태그완성데이터.csv')
df.head()
df.drop(['관광지','관광지명.1'],axis=1, inplace=True)

In [22]:
df.drop_duplicates('관광지명', inplace=True)

In [23]:
df.reset_index(drop=True, inplace=True)

In [24]:
import pandas as pd

# 해시태그, 콤마, 공백을 공백 한 칸으로 전환하는 함수
def clean_hashtags(text):
    text = text.replace(',', '')  # 콤마를 공백으로 대체
    text = text.replace('#', '')  # 해시태그를 공백으로 대체
    text = text.replace('관광지', '') # 관광지 해시태그 제거
    text = text.replace('충청권', '') # 관광지 해시태그 제거
    text = ' '.join(text.split())  # 공백을 한 칸으로 조정
    return text

# 데이터프레임의 해시태그 컬럼에 적용하여 값 변경
df['hashtags'] = df['해시태그'].apply(clean_hashtags)

In [25]:
df

,관광지명,광역시/도,시/군/구,도로명주소,중분류 카테고리,소분류 카테고리,검색건수,언급량,해시태그,가족여부,연인여부,친구여부,아이여부,휴식,교육,hashtags
0,아산스파비스,충청남도,아산시,충남 아산시 아산온천로157번길 67-0,체험관광,웰니스관광,49046,1249,"#가족여행, #관광지, #스파, #아산스파비스, #아이와함께, #연인과함께, #온천...",1,1,0,1,1,0,가족여행 스파 아산스파비스 아이와함께 연인과함께 온천 온천여행 워터파크 휴식공간 휴...
1,파라다이스스파도고,충청남도,아산시,충남 아산시 도고온천로 176-1,체험관광,웰니스관광,35164,1055,"#2023_설날_온천, #가족과함께, #가족여행, #관광지, #데이트코스, #뷰티&...",1,1,0,0,1,0,2023_설날_온천 가족과함께 가족여행 데이트코스 뷰티&스파 스파 실내수영장 야외수...
2,하늘물빛정원,충청남도,금산군,충남 금산군 검한1길 156-0,체험관광,농/산/어촌체험,22495,738,"#가족여행, #걷기길, #관광지, #꽃여행, #먹거리, #미식여행, #사계절, #산...",1,0,0,0,1,0,가족여행 걷기길 꽃여행 먹거리 미식여행 사계절 산책길 산책하기 산책하기좋은곳 야경투...
3,팜카밀레허브농원,충청남도,태안군,충남 태안군 우운길 56-19,체험관광,농/산/어촌체험,21646,515,"#체험, #팜카밀레허브농원, #충청권, #이색체험, #허브, #관광지, #자연환경,...",0,1,0,1,0,0,체험 팜카밀레허브농원 이색체험 허브 자연환경 자연좋은곳 아이와함께 연인과함께 남녀노...
4,아그로랜드태신목장,충청남도,예산군,충남 예산군 상몽2길 231-0,체험관광,농/산/어촌체험,20323,1050,"#2022_추석_당일치기_여행, #가족체험, #관광지, #낙농체험, #아그로랜드태신...",1,0,0,1,1,0,2022_추석_당일치기_여행 가족체험 낙농체험 아그로랜드태신목장 아이가_좋아하는_여...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,사비도성가상체험관,충청남도,부여군,충남 부여군 부소로 15-0,기타관광,기타관광,555,157,#문화시설#사비도성가상체험관#이색체험#가족이색체험#아이와함께#가족과함께#체험프로그램...,1,0,0,1,0,0,문화시설사비도성가상체험관이색체험가족이색체험아이와함께가족과함께체험프로그램VR여행VR체...
393,은포리해안도로,충청남도,보령시,충남 보령시,기타관광,데이트코스,376,9,"#경치좋은곳, #관광지, #남녀노소, #드라이브여행, #드라이브코스, #바다경치, ...",0,1,0,0,1,0,경치좋은곳 남녀노소 드라이브여행 드라이브코스 바다경치 바다내음 연인과함께 해안도로 ...
394,홍성방조제,충청남도,보령시,충남 보령시,기타관광,기타관광,151,451,"#관광지, #바다풍경, #자연, #충청권, #홍성방조제, #힐링",0,0,0,0,0,0,바다풍경 자연 홍성방조제 힐링
395,신비의바닷길,충청남도,보령시,충남 보령시,기타관광,기타관광,99,4901,"#가족여행, #관광지, #모세의기적, #신비의바닷길, #아이와함께, #이색체험, #...",1,0,1,1,0,0,가족여행 모세의기적 신비의바닷길 아이와함께 이색체험 자연 진도신비의바닷길 친구와함께


# 코사인 유사도
- 텍스트라는 칼럼을 추가해서, 해당 부분 전체를 TfidfVectorizer을 이용해서 유사도를 구하고, 이를 바탕으로 추천하는 방식

In [26]:
df['텍스트'] = df['시/군/구'] + ' ' + df['중분류 카테고리'] + ' ' + df['hashtags']

In [27]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['텍스트'])

# 코사인 유사도 계산
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [28]:
# 관광지 추천 함수
def recommend_places(place_name, num_recommendations=10):
    # 입력된 관광지와 동일한 관광지 찾기
    place_index = df[df['관광지명'] == place_name].index[0]

    # 해당 관광지의 코사인 유사도 점수 가져오기
    similarity_scores = list(enumerate(cosine_similarities[place_index]))

    # 코사인 유사도 점수를 기준으로 내림차순 정렬
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # 상위 유사 관광지 인덱스 추출
    top_indices = [score[0] for score in similarity_scores[1:num_recommendations+1]]

    # 상위 유사 관광지 이름 반환
    top_places = df.iloc[top_indices][['관광지명','시/군/구','중분류 카테고리','검색건수','언급량','가족여부','연인여부','친구여부','아이여부']]

    return top_places

In [29]:
# 관광지 추천 예시
place_name = "아산스파비스"  # 추천을 받고자 하는 관광지의 이름 입력
recommendations = recommend_places(place_name)
recommendations

,관광지명,시/군/구,중분류 카테고리,검색건수,언급량,가족여부,연인여부,친구여부,아이여부
9,금강온천,공주시,체험관광,3220,2354,1,0,0,0
1,파라다이스스파도고,아산시,체험관광,35164,1055,1,1,0,0
286,영인산자연휴양림,아산시,자연관광,34855,915,0,0,0,1
254,예당관광지,예산군,문화관광,1559,885,1,0,0,0
208,천안상록리조트,천안시 동남구,문화관광,17664,279,1,0,0,1
306,신두리해수욕장,태안군,자연관광,12607,2913,1,1,1,0
15,만대어촌체험마을,태안군,체험관광,1068,38,1,1,0,0
294,원산도,보령시,자연관광,21108,8138,1,0,0,1
26,서산관광농원,서산시,체험관광,68,323,1,0,0,1
290,파도리해수욕장,태안군,자연관광,23255,3063,1,0,0,0


# 자카드
- 자카드의 경우는 시군구가 제대로 반영이 되지 않는 사항이 발생함

In [30]:
import pandas as pd
import re

def extract_hashtags(text):
    hashtags = re.findall(r"#(\w+)", text)  # 정규표현식을 사용하여 해시태그 추출
    return hashtags

In [31]:
# 데이터프레임에서 모든 해시태그 추출
all_hashtags = []
for tags in df['해시태그']:
    tags_list = tags.split("#")  # 해시태그를 쉼표로 분할하여 리스트로 변환
    all_hashtags.extend(tags_list)  # 리스트에 추가

# 중복 제거를 위해 집합(set)으로 변환 후 다시 리스트로 변환
unique_hashtags = list(set(all_hashtags))

In [32]:
import numpy as np

def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    similarity = intersection / union
    return similarity

# 관광지 수
num_attractions = len(df)

# 데이터프레임 초기화
result_matrix = np.zeros((num_attractions, num_attractions))

# 해시태그 사전 구성
hashtag_dict = {hashtag: i for i, hashtag in enumerate(unique_hashtags)}

# 자카드 유사도 계산 및 결과 저장
for i in range(num_attractions):
    hashtags1 = set(df.loc[i, '해시태그'])
    for j in range(i+1, num_attractions):
        hashtags2 = set(df.loc[j, '해시태그'])
        similarity = jaccard_similarity(hashtags1, hashtags2)
        result_matrix[i, j] = similarity
        result_matrix[j, i] = similarity

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(result_matrix, columns=df['관광지명'], index=df['관광지명'])


In [33]:
# 추천 함수 정의
def recommend_attractions(attraction, top_n=10):
    index = df[df['관광지명'] == attraction].index[0]  # 관광지명으로 인덱스 찾기
    similarities = result_matrix[index]  # 해당 관광지와의 유사도 정보 가져오기
    top_indices = np.argsort(similarities)[::-1][1:top_n+1]  # 유사도가 높은 상위 인덱스 가져오기
    recommended_attractions = df.loc[top_indices, '관광지명']  # 상위 인덱스에 해당하는 관광지명 가져오기
    return recommended_attractions

# 예시: '관광지1'과 유사한 상위 5개 관광지 추천
recommendations = recommend_attractions('아산스파비스', top_n=10)
print(recommendations)

9          금강온천
213     천안삼거리공원
286    영인산자연휴양림
314      안면도수목원
193       청산수목원
26       서산관광농원
254       예당관광지
208     천안상록리조트
27      칠갑산산꽃마을
306     신두리해수욕장
Name: 관광지명, dtype: object


In [34]:
# 1인 값을 0으로 바꾸기 (자카드와 결합 시 사용)
cosine_similarities[np.where(cosine_similarities == 1)] = 0

print(cosine_similarities)

[[0.         0.55122962 0.21361764 ... 0.05866456 0.05387223 0.06274242]
 [0.55122962 0.         0.13003354 ... 0.07811465 0.06046865 0.09899007]
 [0.21361764 0.13003354 0.         ... 0.07267161 0.04169651 0.01988747]
 ...
 [0.05866456 0.07811465 0.07267161 ... 1.         0.23909982 0.11266048]
 [0.05387223 0.06046865 0.04169651 ... 0.23909982 0.         0.14036601]
 [0.06274242 0.09899007 0.01988747 ... 0.11266048 0.14036601 0.        ]]


In [35]:
# 코사인, 자카드의 평균
combine_matrix = (cosine_similarities + result_matrix)/2

In [36]:
# 추천 함수 정의
def recommend_attractions_combine(attraction, top_n=10):
    index = df[df['관광지명'] == attraction].index[0]  # 관광지명으로 인덱스 찾기
    similarities = combine_matrix[index]  # 해당 관광지와의 유사도 정보 가져오기
    top_indices = np.argsort(similarities)[::-1][0:top_n]  # 유사도가 높은 상위 인덱스 가져오기
    recommended_attractions = df.loc[top_indices, '관광지명']  # 상위 인덱스에 해당하는 관광지명 가져오기
    return recommended_attractions

# 예시: '관광지1'과 유사한 상위 5개 관광지 추천
recommendations = recommend_attractions_combine('아산스파비스', top_n=10)
print(recommendations)

9                금강온천
1           파라다이스스파도고
286          영인산자연휴양림
254             예당관광지
213           천안삼거리공원
208           천안상록리조트
26             서산관광농원
306           신두리해수욕장
314            안면도수목원
13     아일랜드리솜오아식스선셋스파
Name: 관광지명, dtype: object


In [37]:
# 코사인 유사도
recommend_places('아산스파비스')

,관광지명,시/군/구,중분류 카테고리,검색건수,언급량,가족여부,연인여부,친구여부,아이여부
1,파라다이스스파도고,아산시,체험관광,35164,1055,1,1,0,0
286,영인산자연휴양림,아산시,자연관광,34855,915,0,0,0,1
254,예당관광지,예산군,문화관광,1559,885,1,0,0,0
208,천안상록리조트,천안시 동남구,문화관광,17664,279,1,0,0,1
306,신두리해수욕장,태안군,자연관광,12607,2913,1,1,1,0
15,만대어촌체험마을,태안군,체험관광,1068,38,1,1,0,0
294,원산도,보령시,자연관광,21108,8138,1,0,0,1
26,서산관광농원,서산시,체험관광,68,323,1,0,0,1
290,파도리해수욕장,태안군,자연관광,23255,3063,1,0,0,0
2,하늘물빛정원,금산군,체험관광,22495,738,1,0,0,0


In [38]:
# 자카드 유사도
recommend_attractions('아산스파비스')

9          금강온천
213     천안삼거리공원
286    영인산자연휴양림
314      안면도수목원
193       청산수목원
26       서산관광농원
254       예당관광지
208     천안상록리조트
27      칠갑산산꽃마을
306     신두리해수욕장
Name: 관광지명, dtype: object

In [39]:
# 둘다 반영
recommend_attractions_combine('아산스파비스')

9                금강온천
1           파라다이스스파도고
286          영인산자연휴양림
254             예당관광지
213           천안삼거리공원
208           천안상록리조트
26             서산관광농원
306           신두리해수욕장
314            안면도수목원
13     아일랜드리솜오아식스선셋스파
Name: 관광지명, dtype: object

In [20]:
result_df

관광지명,아산스파비스,파라다이스스파도고,하늘물빛정원,팜카밀레허브농원,아그로랜드태신목장,아름다운정원화수목,선도리갯벌체험마을,스플라스워터파크,신천탕,금강온천,...,안흥유람선,서산해미읍성회화나무,할미할아비바위,석문국가산업단지음악분수대,코끼리바위,사비도성가상체험관,은포리해안도로,홍성방조제,신비의바닷길,독일마을
관광지명,,,,,,,,,,,,,,,,,,,,,
아산스파비스,0.000000,0.441176,0.380282,0.343284,0.376812,0.438596,0.444444,0.446429,0.254902,0.612245,...,0.105263,0.363636,0.163934,0.155172,0.228571,0.206349,0.400000,0.230769,0.310345,0.282353
파라다이스스파도고,0.441176,0.000000,0.258427,0.238095,0.313253,0.352113,0.312500,0.461538,0.218750,0.367647,...,0.100000,0.236111,0.118421,0.173913,0.176471,0.200000,0.324324,0.200000,0.250000,0.294737
하늘물빛정원,0.380282,0.258427,0.000000,0.238095,0.297619,0.297297,0.329114,0.233766,0.130435,0.388060,...,0.084507,0.236111,0.133333,0.109589,0.250000,0.097561,0.256410,0.218750,0.184211,0.182692
팜카밀레허브농원,0.343284,0.238095,0.238095,0.000000,0.383562,0.442623,0.366197,0.208333,0.147541,0.231884,...,0.061538,0.285714,0.132353,0.123077,0.260274,0.281250,0.363636,0.206897,0.322581,0.321839
아그로랜드태신목장,0.376812,0.313253,0.297619,0.383562,0.000000,0.409091,0.500000,0.210526,0.153846,0.323529,...,0.138462,0.283582,0.171429,0.130435,0.276316,0.260870,0.319444,0.190476,0.338462,0.290323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
사비도성가상체험관,0.206349,0.200000,0.097561,0.281250,0.260870,0.213115,0.220588,0.158730,0.120000,0.092308,...,0.078431,0.288462,0.145455,0.113208,0.130435,0.000000,0.225806,0.056604,0.307692,0.174419
은포리해안도로,0.400000,0.324324,0.256410,0.363636,0.319444,0.344262,0.358209,0.208955,0.185185,0.316667,...,0.086207,0.250000,0.183333,0.175439,0.303030,0.225806,0.000000,0.185185,0.245902,0.252874
홍성방조제,0.230769,0.200000,0.218750,0.206897,0.190476,0.169811,0.203390,0.196078,0.128205,0.255319,...,0.023810,0.222222,0.133333,0.093023,0.222222,0.056604,0.185185,0.000000,0.191489,0.155844
